In [1]:
import torch
import sys
import yaml
from torchvision import transforms, datasets
import torchvision
import numpy as np
import os
from sklearn import preprocessing
from torch.utils.data.dataloader import DataLoader

In [2]:
sys.path.append('../')
from models.pointnetv2_encoder import PointNetV2

In [3]:
batch_size = 10

In [4]:
config = yaml.load(open("../config/config.yaml", "r"), Loader=yaml.FullLoader)

In [5]:
# from data_utils.ModelNetDataLoader import ModelNetDataLoader
# train_dataset = ModelNetDataLoader('/data/nerdxie/neuron_cluster/neuron_dataset_1',split='train', uniform=True, normal_channel=True,istrain=False,)
# test_dataset = ModelNetDataLoader('/data/nerdxie/neuron_cluster/neuron_dataset_1',split='test', uniform=True, normal_channel=True,istrain=False,)
from data_utils.FAFBAnalysisDataLoader import TestDataLoader
# full_dataset = ModelNetDataLoader('./data_neuron.h5', uniform=True, normal_channel=True, istrain=False)
test_dataset = TestDataLoader('/data/nerdxie/neuron_cluster/neuron_dataset_add_all_128/',split='train', uniform=True, normal_channel=True,istrain=False,)

------------------- True
The size of train data is 668782


In [6]:
test_loader = DataLoader(test_dataset, batch_size=batch_size,
                          num_workers=4, drop_last=True, shuffle=True)

In [7]:
device = 'cuda' #'cuda' if torch.cuda.is_available() else 'cpu'
encoder = PointNetV2(**config['network']).to(device)
# encoder.projetion = torch.nn.Sequential()
encoder = torch.nn.DataParallel(encoder).cuda()

In [8]:
#load pre-trained parameters

# for i in load_params['online_network_state_dict'].keys():
#     if 'projetion' in i:
#         del load_params['online_network_state_dict'][i]

encoder.load_state_dict(torch.load(os.path.join('encoder2.pt'),
                         map_location=torch.device(torch.device(device))))
print("Parameters successfully loaded.")

# remove the projection head
# encoder = torch.nn.Sequential(*list(encoder.cpu().children())[:-1])    
# encoder = encoder.to(device)
# output_feature_dim = 1024

Parameters successfully loaded.


In [9]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)

In [10]:
device = 'cuda'
logreg = LogisticRegression(1024, 3)
logreg = logreg.to(device)
logreg.load_state_dict(torch.load("linear2.pt"))

<All keys matched successfully>

In [11]:
def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []
    names = []

    # get the features from the pre-trained model
    for batch, y, index in loader:
        index = [int(i) for i in index]
        with torch.no_grad():
            batch = batch.to(device)
            features = encoder(batch)[1]
            x_train.extend(features)
            y_train.extend(y.numpy())
            names.extend(index)

            
    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    names = torch.tensor(names)
    return x_train, y_train, names

In [12]:
from itertools import chain
logreg.eval()
encoder.eval()
pre = []
t_file = '/code/PyTorch-BYOL/eval/predicted_glia_original_data.txt'
t = open(t_file, "w")
names = []
for batch, y, index in test_loader:
    index = [int(i) for i in index]
    with torch.no_grad():
        batch = batch.to(device)
        features = encoder(batch)[1]
        logits = logreg(features)
        # print(logits)
        predictions = torch.argmax(logits, dim=1)
        # print(predictions)
        pre.extend(predictions)
        names.extend(index)
        glias = np.where(predictions.cpu() == 1)
        for glia in glias[0]:
            t.write(str(index[glia])+',\n')
            print(str(index[glia])+',\n')

7373261686,

6728294998,

4773759577,

5152613609,

1364923236,

7123331250,

3339227829,

3476793718,

4854978392,

5239341559,

7786366204,

9038233181,

9966795344,

7007222160,

3594623136,

5007297358,

6833507315,

7102001756,

9728463636,

5575718128,

3312459596,

5537111588,

8115709146,

7219284669,

3048185702,

11668962620,

3945788599,

10122552546,

9593485909,

6316222667,

9596019497,

7902560961,

1889518298,

1511523325,

7886967662,

9590526182,

5668514759,

9590460255,

11163690782,

5004796152,

8169935449,

9702933988,

7258364614,

2279222153,

4856802544,

11429012371,

6087899674,

8404141761,

6340375788,

7062710568,

10634710896,

5137571488,

5579319116,

5251453421,

5048149262,

2026537380,

7753230466,

6366433664,

6192030877,

1993267566,

5402532419,

7007456071,

3984958787,

8147757316,

4097605359,

9607258826,

4984684651,

7014547076,

8409556950,

1743004132,

1496097455,

4658070385,

12196788978,

3717968191,

4489018209,

2942043617,

972781

KeyboardInterrupt: 

In [41]:
glias

(array([6, 9]),)